In [36]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import gc

import string

from tqdm import tqdm

import pickle

from sklearn.metrics import fbeta_score


In [37]:
## store the embeddings and one hot encodings using np.save()

In [38]:
X_test,y_test,X_train,y_train = np.load('/kaggle/input/dal-train/embed_train.npy'),np.load('/kaggle/input/dal-train/labels_train.npy'),np.load('/kaggle/input/dal-train/embed_test.npy'),np.load('/kaggle/input/dal-train/labels_test.npy')

In [39]:
X_train = np.vstack((X_train,X_test))

y_train = np.vstack((y_train,y_test))

In [40]:
alphabets = list(string.ascii_uppercase)

In [41]:
# store both the multi label binarizer models

In [42]:
with open('/kaggle/input/dal-train/mlb.pkl','rb') as file:

    mlb = pickle.load(file)

with open('/kaggle/input/dal-train/mlb_letter.pkl','rb') as file:

    mlb_letter = pickle.load(file)

In [43]:
#y_train_letter

y_labels_train = mlb.inverse_transform(y_train)

y_labels_test = mlb.inverse_transform(y_test)

In [44]:
y_starter_train = [tuple(s[0] for s in tup) for tup in y_labels_train]

y_starter_test = [tuple(s[0] for s in tup) for tup in y_labels_test]

In [47]:
y_train_letter = mlb_letter.transform(y_starter_train)

y_test_letter = mlb_letter.transform(y_starter_test)

In [48]:
gc.collect()

0

In [49]:
import tensorflow as tf
from tensorflow.keras import backend as K

def micro_f2(y_true, y_pred):
    y_true = K.cast(y_true, tf.float32)
    y_pred = K.cast(y_pred > 0.5, tf.float32)  # Threshold to get binary predictions
    true_positives = K.sum(y_true * y_pred)
    possible_positives = K.sum(y_true)
    predicted_positives = K.sum(y_pred)
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f2 = (1 + 2**2) * (precision * recall) / (2**2 * precision + recall + K.epsilon())
    return f2




In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def build_model(input_length, num_labels):
    model = models.Sequential()

    # Word Embedding Layer dim 128

    # Bidirectional GRU Layer
    model.add(layers.Bidirectional(layers.GRU(256, return_sequences=True)))

    # Fully Connected Layer 1
    model.add(layers.Flatten())  # Flattening the output from the GRU layer
    model.add(layers.Dense(700, activation='relu'))
    model.add(layers.Dropout(0.2))

    # Fully Connected Layer 2 (Output Layer)
    model.add(layers.Dense(num_labels, activation='sigmoid'))  # Sigmoid for multi-label classification

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[micro_f2])

    return model





In [51]:
# first stage for letters
model1 = build_model(X_train.shape[1],y_train_letter.shape[1])
model1.fit(X_train.reshape(X_train.shape[0],1,X_train.shape[1]),y_train_letter, epochs = 10, batch_size = 64)

Epoch 1/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0425 - micro_f2: 0.8540
Epoch 2/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0215 - micro_f2: 0.9245
Epoch 3/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0189 - micro_f2: 0.9331
Epoch 4/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0167 - micro_f2: 0.9416
Epoch 5/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0151 - micro_f2: 0.9463
Epoch 6/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0136 - micro_f2: 0.9518
Epoch 7/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0123 - micro_f2: 0.9563
Epoch 8/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0113 - micro_f2: 0.9604
Epoch 9/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0104 - micro_f2: 0.9634
Epoch 10/10
3060/3060 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0096 - micro_f2: 0.9666


In [52]:
def filter_data_points(embeddings,labels, labels_to_check):
    #choose rows who have atleast one entry in 'labels_to_check' and choose labels data only from 'labels_to_check' columns
    interested_rows = np.any(labels[:, labels_to_check], axis=1)
    interested_labels = labels[interested_rows]
    filtered_labels = np.zeros((np.sum(interested_rows),labels.shape[1]))
    filtered_labels[:,labels_to_check] += interested_labels[:,labels_to_check] 
    filtered_embeddings = embeddings[interested_rows]
    return filtered_embeddings, filtered_labels

In [53]:
#Training a model for each alphabet
mlabels = mlb.classes_
models2 = {}
for letter in tqdm(alphabets):
    # X should be embeddings, y should be multilabels
    indices  = [ind for ind,s in enumerate(mlabels) if s.startswith(letter)]
    filtered_data,filtered_labels = filter_data_points(X_train,y_train,indices)
    if len(indices) == 0:
        models2[letter] = []
        continue
    elif len(indices) == 1:
        mask = np.zeros((1,len(mlabels)))
        mask[0,indices[0]] = 1
        models2[letter] = mask
    else:
        model = build_model(filtered_data.shape[1],len(indices))
        model.fit(filtered_data.reshape(filtered_data.shape[0],1,filtered_data.shape[1]),filtered_labels[:,indices],epochs = 15)
        models2[letter] = model

  0%|          | 0/26 [00:00<?, ?it/s]

Epoch 1/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3377 - micro_f2: 0.6854
Epoch 2/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0698 - micro_f2: 0.9175
Epoch 3/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0389 - micro_f2: 0.9532
Epoch 4/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0386 - micro_f2: 0.9566
Epoch 5/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0427 - micro_f2: 0.9432
Epoch 6/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0367 - micro_f2: 0.9525
Epoch 7/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0373 - micro_f2: 0.9534
Epoch 8/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0180 - micro_f2: 0.9835
Epoch 9/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0158 - micro_f2: 0.9824
Epoch 10/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0130 - micro_f2: 0.9935
Epoch 11/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0153 - micro_f2: 0.9822
Epoch 12/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0071 - micro_

  8%|▊         | 2/26 [00:04<00:51,  2.16s/it]

Epoch 1/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0988 - micro_f2: 0.5240
Epoch 2/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0151 - micro_f2: 0.8691
Epoch 3/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0119 - micro_f2: 0.8976
Epoch 4/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0108 - micro_f2: 0.9085
Epoch 5/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0089 - micro_f2: 0.9221
Epoch 6/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0077 - micro_f2: 0.9349
Epoch 7/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0067 - micro_f2: 0.9423
Epoch 8/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0057 - micro_f2: 0.9495
Epoch 9/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0052 - micro_f2: 0.9561
Epoch 10/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0049 - micro_f2: 0.9614
Epoch 11/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0047 - micro_f2: 0.9593
Epoch 12/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

 12%|█▏        | 3/26 [00:26<04:01, 10.50s/it]

...

 73%|███████▎  | 19/26 [11:30<02:57, 25.42s/it]

Epoch 1/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1441 - micro_f2: 0.4806
Epoch 2/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0438 - micro_f2: 0.7540
Epoch 3/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0363 - micro_f2: 0.7896
Epoch 4/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0308 - micro_f2: 0.8372
Epoch 5/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0272 - micro_f2: 0.8512
Epoch 6/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0243 - micro_f2: 0.8736
Epoch 7/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0205 - micro_f2: 0.8866
Epoch 8/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0178 - micro_f2: 0.9084
Epoch 9/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0153 - micro_f2: 0.9149
Epoch 10/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0126 - micro_f2: 0.9357
Epoch 11/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0114 - micro_f2: 0.9448
Epoch 12/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

 77%|███████▋  | 20/26 [11:45<02:15, 22.51s/it]

Epoch 1/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.0351 - micro_f2: 0.7294
Epoch 2/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0082 - micro_f2: 0.8882
Epoch 3/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0068 - micro_f2: 0.9066
Epoch 4/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0055 - micro_f2: 0.9263
Epoch 5/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0046 - micro_f2: 0.9377
Epoch 6/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0040 - micro_f2: 0.9478
Epoch 7/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0034 - micro_f2: 0.9551
Epoch 8/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0030 - micro_f2: 0.9605
Epoch 9/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0027 - micro_f2: 0.9652
Epoch 10/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0023 - micro_f2: 0.9702
Epoch 11/15
1100/1100 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0022 - micro_f2: 0.9720
Epoch 12/15
1100/1100 ━━━━━━━━

100%|██████████| 26/26 [13:20<00:00, 30.80s/it]


In [54]:
def final_pred(X_test,pred_1,mlabels,models):
    pred_2 = np.zeros((X_test.shape[0],len(mlabels)))
    ind = 0
    for letter in tqdm(alphabets):
        indices  = [ind for ind,s in enumerate(mlabels) if s.startswith(letter)]

        if letter == 'P' or letter == 'U':
            continue
        reqd_data = np.any(pred_1[:,ind].reshape(-1,1),axis = 1)  #all rows where the pred_1 is 1 for the given letter
        reqd_model = models[letter]
        ind +=1
        if np.sum(reqd_data) == 0:
            continue
        elif isinstance(reqd_model,type(np.array([]))):
            pred_2[reqd_data,:] +=reqd_model
        else:
            pred_2[np.ix_(reqd_data,indices)] += reqd_model.predict(X_test[reqd_data,:])>0.5
    return pred_2

In [55]:
gc.collect()

8929

In [56]:
real_test = np.load('/kaggle/input/dal-train/test_data.npy')

In [57]:
pred_1 = model1.predict(real_test.reshape(real_test.shape[0],1,real_test.shape[1]))>0.5

3110/3110 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step


In [58]:
pred_2 = mlb.inverse_transform(final_pred(real_test.reshape(real_test.shape[0],1,real_test.shape[1]),pred_1,mlabels,models2))

  0%|          | 0/26 [00:00<?, ?it/s]

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


  8%|▊         | 2/26 [00:00<00:04,  4.88it/s]

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


 12%|█▏        | 3/26 [00:00<00:06,  3.67it/s]

...

 73%|███████▎  | 19/26 [00:15<00:06,  1.15it/s]

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


 77%|███████▋  | 20/26 [00:16<00:04,  1.31it/s]

1015/1015 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


100%|██████████| 26/26 [00:19<00:00,  1.34it/s]


In [59]:
empty_pred = [i for i,pred in enumerate(pred_2) if pred == ()]

In [60]:
len(empty_pred)

2695

In [61]:
import csv

filename = "output_13.csv"



# Write the list of tuples to the CSV file

with open(filename, mode='w', newline='') as file:

    writer = csv.writer(file)

    # Write the header

    writer.writerow(["id", "labels"])

    

    # Write each row with sequential ID

    for idx, labels in enumerate(pred_2, start=1):

        # Join labels with ';' and write to file

        writer.writerow([idx, ";".join(labels)])



print(f"Data saved to {filename}")

Data saved to output_13.csv


In [55]:
final_pred

<function __main__.final_pred(X_test, pred_1, means, mlabels, models_dict)>

In [ ]:
gc.collect()

In [ ]:
# pred_2 = final_pred(X_test[:n_test],y_cluster_no,means,mlabels,models_dict)

In [ ]:
# print(f"Score is {fbeta_score(pred_2,y_test[:n_test],beta = 2,average = 'micro')}")

In [ ]:
### A few comparisons ###

In [ ]:
mlb.inverse_transform(pred_2[:5])

In [ ]:
y_labels_test[:5]

In [ ]:
# means[np.where((means>=0.1) * (means<=1.00))[0],np.where((means>=0.1) * (means<=1.00))[1]] = 1

# means[np.where(means!=1)] = 0